In [1]:
# %matplotlib inline
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.font_manager
import pandas as pd

from scipy import stats
import itertools

from sklearn import svm
from sklearn.covariance import EmpiricalCovariance, MinCovDet
from sklearn.decomposition import PCA

import csv
import datetime
import dateutil.parser
import netCDF4
import os
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
#Input the file name.
fname = 'hotspot_2015'

fire = pd.read_csv(fname+".csv", parse_dates=['datetime'], usecols=['latitude','longitude','datetime', 'power'])
#Skip the 1st header row.
#data.next()
fire.head()

,latitude,longitude,datetime,power
0,-15.0460,144.3795,2015-01-01 01:04:50,14.2
1,-15.8927,136.6096,2015-01-01 01:00:50,11.9
2,-17.1466,145.8501,2015-01-01 01:00:50,10.0
3,-18.0796,122.6971,2015-01-01 02:41:30,11.1
4,-18.0764,122.6902,2015-01-01 02:45:40,11.0


In [3]:
def get_data_from_netcdf(latitude,longitude,fire_date,window):
    # dictionary for met data
    met_data = {}
    # initialise constants
    latitude0 = -9.75
    longitude0 = 109.5
    lat_resolution = 0.75
    lon_resolution = 0.75
    nc_file_path = "/Volumes/UNTITLED/peter/"
    # get the year
    year = fire_date.year
    # get the latitude and longitude indices
    lat_index = int(((latitude0-latitude)/lat_resolution)+0.5)
    lon_index = int(((longitude-longitude0)/lon_resolution)+0.5)
    # get the name of the netCDF file containing the data
    nc_file_name = str(year)+"_"+str(lat_index)+"_"+str(lon_index)+".nc"
    nc_full_name = nc_file_path+nc_file_name
    print (nc_full_name)
    if not os.path.exists(nc_full_name):
#         print (" File "+nc_full_name+" not found, skipping ...")
        return met_data
    # get the data
    print (nc_file_name)
    nc_file = netCDF4.Dataset(nc_full_name,'r')
    nc_time = nc_file.variables["time"][:]
    nc_time_units = getattr(nc_file.variables["time"],"units")
    dt_utc = netCDF4.num2date(nc_time,nc_time_units)
    start_date = fire_date - datetime.timedelta(days=window["before"])
    end_date = fire_date + datetime.timedelta(days=window["after"])
    if start_date<datetime.datetime(year,1,1,0,0,0):
#         print (" start_date before beginning of year, skipping ...")
        return met_data
    si = [n for n,dt in enumerate(dt_utc) if dt>start_date ][0]
    ei = [n for n,dt in enumerate(dt_utc) if dt>end_date ][0]
    met_data["DateTime"] = dt_utc[si:ei+1]
    met_data["Fsd"] = nc_file.variables["Fsd"][si:ei+1]
    met_data["Ta"] = nc_file.variables["Ta"][si:ei+1]
    met_data["RH"] = nc_file.variables["RH"][si:ei+1]
    met_data["Ws"] = nc_file.variables["Ws"][si:ei+1]
    met_data["Wd"] = nc_file.variables["Wd"][si:ei+1]
    met_data["Precip"] = nc_file.variables["Precip"][si:ei+1]
    nc_file.close()
    return met_data

In [4]:
print( fire.latitude[0])

-15.046


In [ ]:
year = 2015
window = {"before":5,"after":0}
array = np.empty([len(fire.latitude), 7])
for i, fid in enumerate(fire.latitude):
    if fire.power[i]<50: continue 
    latitude = fire.latitude[i]
    longitude = fire.longitude[i]
    fire_date = fire.datetime[i]
    met_data = get_data_from_netcdf(latitude,longitude,fire_date,window)
    if "DateTime" not in met_data: continue
    print(latitude,longitude,[np.mean(met_data['Ta']), np.mean(met_data['Fsd']), np.mean(met_data['Ws']), np.mean(met_data['Wd']), np.mean(met_data['Precip']) ])
    array[i] = [latitude,longitude,np.mean(met_data['Ta']), np.mean(met_data['Fsd']), np.mean(met_data['Ws']), np.mean(met_data['Wd']), np.mean(met_data['Precip']) ]
    


/Volumes/UNTITLED/peter/2015_16_9.nc
2015_16_9.nc
/Volumes/UNTITLED/peter/2015_16_9.nc
2015_16_9.nc
/Volumes/UNTITLED/peter/2015_16_9.nc
2015_16_9.nc
/Volumes/UNTITLED/peter/2015_16_9.nc
2015_16_9.nc
/Volumes/UNTITLED/peter/2015_16_9.nc
2015_16_9.nc
/Volumes/UNTITLED/peter/2015_16_9.nc
2015_16_9.nc
/Volumes/UNTITLED/peter/2015_16_9.nc
2015_16_9.nc
/Volumes/UNTITLED/peter/2015_16_9.nc
2015_16_9.nc
/Volumes/UNTITLED/peter/2015_16_9.nc
2015_16_9.nc
/Volumes/UNTITLED/peter/2015_16_9.nc
2015_16_9.nc
/Volumes/UNTITLED/peter/2015_17_53.nc
2015_17_53.nc
/Volumes/UNTITLED/peter/2015_14_52.nc
2015_14_52.nc
/Volumes/UNTITLED/peter/2015_11_48.nc
/Volumes/UNTITLED/peter/2015_9_47.nc
/Volumes/UNTITLED/peter/2015_8_48.nc
/Volumes/UNTITLED/peter/2015_6_46.nc
2015_6_46.nc
/Volumes/UNTITLED/peter/2015_16_9.nc
2015_16_9.nc
/Volumes/UNTITLED/peter/2015_8_46.nc
/Volumes/UNTITLED/peter/2015_7_48.nc
/Volumes/UNTITLED/peter/2015_8_45.nc
2015_8_45.nc
/Volumes/UNTITLED/peter/2015_8_46.nc
/Volumes/UNTITLED/peter

In [ ]:
array.shape()

In [ ]:
#sample paarmeters

n_inliers = 100
n_outliers = 20
outliers_fraction=0.2

# Data generation
X1 = 2 * np.random.randn( n_inliers, 3)-6
X2 = 3 * np.random.randn(n_inliers, 3) +6
X = np.r_[X1, X2]
# print(X.shape)
# Add outliers
X = np.r_[X, np.random.uniform(low=-6, high=6, size=(n_outliers, 3))]

# Add longitude and latitude
longitude=2 * np.random.randn( 2*n_inliers+n_outliers, 1)-6
latitude = 2 * np.random.randn( 2*n_inliers+n_outliers, 1)-6


X_all =np.c_[X,longitude,latitude]

# print(X_location)

In [ ]:
X_raw= X_all[:,0:3]
X_locations=X_all[:,3:5]
# print(X_raw)
###### pca
pca = PCA(n_components=2)
X=pca.fit_transform(X_raw)
# print(Y_pca)
# print(pca.explained_variance_ratio_) 

In [ ]:
model=svm.OneClassSVM(nu=0.95 * outliers_fraction + 0.05,  kernel="rbf", gamma=0.1).fit(X)
print(model)
# eliptic envalope
# model= EllipticEnvelope(contamination=.1).fit(X)

In [ ]:
#predicted score of a point not being an outlier
y_pred = model.decision_function(X).ravel()
# print(y_pred)

In [ ]:
##potentially we can let user to change the treshhold
threshold = stats.scoreatpercentile(y_pred,100 * outliers_fraction)

# threshold=0.6
print(threshold)

In [ ]:
y_pred = y_pred > threshold
# print(y_pred)

In [ ]:
X_new = np.c_[X, y_pred,X_locations]
# print(min(X[:,1]))
# print(X_new)

In [ ]:
#plotting the stuff
marg=5
xx, yy = np.meshgrid(np.linspace(min(X[:,0])-marg, max(X[:,0])+marg, 500), np.linspace(min(X[:,1])-marg, max(X[:,1])+marg, 500))
Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.figure()

#contour lines for outlier detection
plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), threshold, 7),cmap=plt.cm.Blues_r)
plt.contourf(xx, yy, Z, levels=[threshold, Z.max()],colors='orange')

#scatterplot of the data
inliers = X_new[X_new[:, 2] == 1]
plt.scatter(inliers[:, 0], inliers[:, 1], color='black', label='inliers')

outliers = X_new[X_new[:, 2] == 0]
plt.scatter(outliers[:, 0], outliers[:, 1], color='red', label='inliers')
plt.axis('off')
plt.show()

In [ ]:
outliers_location=outliers[:,3:5]
# print(outliers_location.shape)
out_loc_pd=pd.DataFrame(outliers_location)
out_loc_pd.columns = ['latitude', 'longitude']
# print(out_loc_pd)
outlier_json=out_loc_pd.to_json(orient="records")

inliers_location=outliers[:,3:5]
inl_loc_pd=pd.DataFrame(outliers_location)
inl_loc_pd.columns = ['latitude', 'longitude']

inlier_json=inl_loc_pd.to_json(orient="records")
# print(inl_loc_pd)